In [1]:
# Project Overview: Create Relevant Spotify Playlist with New Songs
# 0. Declare name of playlist to be created
# 1. Import Packages and establish creds to connect to Spotify
# 2. Get raw Spotify data of Liked (Hearted) songs into a dataframe
# 3. Add audio features
# 4. Add genre data
# 5. Create genre score
### Repeat process (steps 2-5) for disliked songs (based on dislike playlist) ###
# 6. Create playlist df (to access disliked playlist)


In [24]:
# 0. Declare name of playlist to be created
pred_like_playlist_name = "Genre_Score>=50"

In [3]:
#### ALL FUNCTIONS BELOW ####

# 1. Import Packages and establish creds to connect to Spotify
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import requests
import math
import time
import pandas as pd
from IPython.display import display
from pandas.io.json import json_normalize
import numpy as np 
import matplotlib.pyplot as plt  
import inspect
import pickle

def define_scope():
    global username
    username = os.environ['SPOTIPY_USERNAME']
    global scope 
    scope = '''
    playlist-modify-private 
    playlist-modify-public 
    playlist-read-collaborative 
    playlist-read-private
    user-follow-modify 
    user-follow-read 
    user-library-modify 
    user-library-read 
    user-modify-playback-state 
    user-read-currently-playing 
    user-read-email 
    user-read-playback-state 
    user-read-private 
    user-read-recently-played 
    user-top-read'''
    global token, sp, user, headers
    try:
        token = util.prompt_for_user_token(username, scope)
    except (AttributeError, JSONDecodeError):
        os.remove(f".cache-{username}")
        token = util.prompt_for_user_token(username, scope)
    sp = spotipy.Spotify(auth=token)
    user = sp.current_user()
    headers = {'Authorization': "Bearer {}".format(token)}
    return

# 2. Get raw Spotify data of Liked (i.e. Hearted) songs into a dataframe
# 2a. Get raw data of liked songs into a list
def req(url,headers):
    r = requests.get(url, headers=headers)
    return json.loads(r.text)

def get_raw_liked_song_list():
    print('Get_raw_liked_song_list')
    offset = 0
    url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}"

    liked_song_count = req(url,headers=headers)["total"]
    print("Liked song count: ", liked_song_count)

    raw_liked_songs = []
    for i in range(int(math.ceil(liked_song_count/50.0))):
        offset = 50*i
        raw_liked_songs.extend(req(f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}",headers=headers)["items"])

    print(f">> raw liked songs['0'] \n>> {raw_liked_songs[0]}")
    return raw_liked_songs

# 2b. transform liked song raw data list into df
# parse out data from Liked Songs' raw data
def song_metadata_to_df(raw_songs):
    print('Transforming song raw data list into df\n')
    define_scope()
    # create empty df
    df = pd.DataFrame(columns = [
        'album', 
        'artist_uri', 
        'artist',
        'track',
        'url',
        'popularity',
        'added_at',
        'release_date',
        'release_year',
        'uri'
        ])
    # parse out songs 100 at a time
    for i in range(0, len(raw_songs)):
        if (i/100).is_integer():
            print(f"{i}/{len(raw_songs)} parsed")
        try:
            album = raw_songs[i]['track']['album']['name']
            artist_uri = raw_songs[i]['track']['artists'][0]['uri'].split(':')[2]
            artist = raw_songs[i]['track']['album']['artists'][0]['name']
            track = raw_songs[i]['track']['name']
            url = raw_songs[i]['track']['external_urls']['spotify']
            popularity = raw_songs[i]['track']['popularity']
            added_at = raw_songs[i]['added_at']
            release_date = raw_songs[i]['track']['album']['release_date']
            release_year = release_date.split('-')[0]
            uri = raw_songs[i]['track']['uri']
            df = df.append(pd.DataFrame([[
                album, 
                artist_uri,
                artist, 
                track, 
                url, 
                popularity, 
                added_at, 
                release_date, 
                release_year,
                uri
                ]], columns = [
                'album', 
                'artist_uri', 
                'artist',
                'track',
                'url',
                'popularity',
                'added_at',
                'release_date',
                'release_year',
                'uri'
                ]))
        except:
            pass
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

#3. Add audio features 
# Combine liked song df with audio feature df
def add_audio_feats(df):
    print('Combining song_df with audio_feature_df')
    # get list of song uris
    song_uri_list = df.index.to_list()
    audio_feature_df = pd.DataFrame()
    for i in range(0,math.ceil(len(song_uri_list)/100)):
        define_scope()
        features = sp.audio_features(song_uri_list[(i)*100:100*(i+1)])
        audio_feature_df = audio_feature_df.append(pd.DataFrame(features))
    audio_feature_df.set_index('uri', inplace=True)
    # merge this 
    liked_song_df = df.merge(audio_feature_df, on = 'uri')
    display(liked_song_df.head())
    return liked_song_df.drop_duplicates()

#4. Add genre data
# 4a. Get artist_uris -> get genres
def get_genres(df):
    print('Getting genres\n')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(len(artist_uris), ' distinct artists')
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            print(f"{i}/{len(artist_uris)} parsed")
        try:
            g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
        except:
            define_scope()
            try:
                g.append(req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)['genres'])
            except:
                pass
    # genres_df's artist_uri column stores a list of genres per artist uri
    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    
    print('\ngenre_df')
    display(genres_df.head())
    return genres_df

# There is a list of genres for one artist that must be broken out
def explode_genres(df):
    genre_exploded_df = df.explode('genre list')
    
    print('\ngenre_exploded_df')
    display(genre_exploded_df.head())
    return genre_exploded_df

# Add these genres back to the df for each song
def add_genres(df, genre_exploded_df):
    # the following does not work oddly enough
    # liked_song_df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri', inplace = True)
    df = df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri')
    df.rename(columns={'genre list': 'genre'}, inplace=True)
    
    print('Merging genres into song_df')
    display(df.head())
    return df

#5. Create Genre Score

# Get genre counts for liked songs - the higher the count, the higher the score
def get_genre_counts(genre_exploded_df):
    print('Getting genre counts')
    # establish genre counts (which will determine score)
    genre_count_df = pd.DataFrame(genre_exploded_df['genre list'].value_counts()).reset_index()
    genre_count_df.columns = ['genre','genre count']
    print('\ngenre_count_df.head()')
    display(genre_count_df.head())
    return genre_count_df

# Merge the genre counts with the song df and sum the genre column
def establish_genre_score(genre_count_df, df):
    print('\nMerging genre counts with song_df')
    df = df.reset_index().merge(genre_count_df , on = 'genre',how = 'left')
    df.drop(columns=['genre'], inplace=True)
    display(df.head())

    groupby_cols = df.columns.to_list()[:-1]
    
    print('\nSumming genre column')
    df = df.groupby(groupby_cols).sum()
    df = df.rename({'genre count': 'genre score'}, axis=1).reset_index() 
    # reset index after summing - otherwise everything prior to sum will become index
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

# 6. Create playlist df (to access disliked playlist)
# get playlist raw data (for dislikes)
def get_playlist_uris(playlists_raw):
    df = pd.DataFrame()
    for i in range(0, len(playlists_raw)):
        playlist = playlists_raw['items'][i]['name']
        playlist_uri = playlists_raw['items'][i]['uri']
        song_count = playlists_raw['items'][i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    print('playlist_df')
    display(df)
    return df

def get_raw_disliked_song_list():
    playlists_raw = sp.current_user_playlists()
    playlists_df = get_playlist_uris(playlists_raw)
    uri =  playlists_df.loc['Dislikes']['playlist_uri']
    song_count =  playlists_df.loc['Dislikes']['song_count']
    # extract data from Disliked songs
    dislike_raw_list = []
    for i in range(int(math.ceil(song_count/100.0))):
        dislike_raw_list.extend(sp.user_playlist_tracks(uri.split(':')[1], uri.split(':')[2], offset=i*100)['items'])
    print('Getting raw disliked song list')
    return dislike_raw_list

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
def combine_dfs_to_create_my_songs_df(df1,df2, save_destination):
    print('Combining dfs to create my_songs_df and storing as pickle file')
    df1['Liked'] = 1
    df2['Liked'] = 0
    my_songs_df = pd.concat([df1,df2])
    my_songs_df.to_csv(f"{save_destination}/my_songs.csv")
    my_songs_df.to_pickle("./my_songs.pkl")
    display(my_songs_df.head())
    print('successfully saved to .csv and .pkl files in save_destination')

# 8. Get featured playlist uris so we can get their associated songs
def get_featured_playlist_uris():
    print('Getting raw_featured_playlist_df')
    url = f"https://api.spotify.com/v1/browse/featured-playlists?limit=50&offset=0"
    featured_playlist_count = req(url,headers=headers)['playlists']['total']
    df = pd.DataFrame()
    for i in range(0,featured_playlist_count):
        raw_featured_playlist = req(url,headers=headers)
        playlist = raw_featured_playlist['playlists']['items'][i]['name']
        playlist_uri = raw_featured_playlist['playlists']['items'][i]['uri']
        song_count = raw_featured_playlist['playlists']['items'][i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

# 9. Get raw data from the songs
def get_raw_featured_playlist_song_list():
    playlists_df = get_featured_playlist_uris()
    raw_featured_songs = []
    for uri in playlists_df.loc[:,'playlist_uri'].to_list():
        song_count = playlists_df.loc[playlists_df['playlist_uri']==uri,'song_count'][0]
        offset = 0
        url = f"https://api.spotify.com/v1/playlists/{uri.split(':')[2]}/tracks?limit=50&offset={offset}"
        for i in range(int(math.ceil(song_count/100.0))):
            offset = 100*i
            raw_featured_songs.extend(req(url,headers=headers)['items'])

    return raw_featured_songs

def create_playlist(pred_like_playlist_name, df, genre_score_threshold):
    playlists_raw = sp.current_user_playlists()
    playlists_df = get_playlist_uris(playlists_raw)
    playlist_uri = get_playlist_uris(playlists_raw).loc[playlists_df.index == pred_like_playlist_name,'playlist_uri'][0].split(':')[2]
    uri_list = df.loc[df['genre score']>= genre_score_threshold].reset_index()['uri'].to_list()
    num_songs_to_add = len(uri_list)
    for i in range(int(math.ceil(num_songs_to_add/100.0))):
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={','.join(uri_list[i*100:(i+1)*100])}"
        requests.post(url, headers=headers)
    return

In [4]:
define_scope()
print('Scope defined\n****PART 1: Get Liked Songs Dataframe****\n')
raw_liked_song_list = get_raw_liked_song_list()
liked_song_df = song_metadata_to_df(raw_liked_song_list)
liked_song_df = add_audio_feats(liked_song_df)
genre_df = get_genres(liked_song_df)
genre_exploded_df = explode_genres(genre_df)
liked_song_df = add_genres(liked_song_df, genre_exploded_df)
liked_genre_count_df = get_genre_counts(genre_exploded_df)
liked_song_df = establish_genre_score(liked_genre_count_df, liked_song_df)
print('\n****PART 2: Get Disliked Songs Dataframe****\n')
define_scope()
raw_disliked_song_list = get_raw_disliked_song_list()
disliked_song_df = song_metadata_to_df(raw_disliked_song_list)
disliked_song_df = add_audio_feats(disliked_song_df)
genre_df = get_genres(disliked_song_df)
genre_exploded_df = explode_genres(genre_df)
disliked_song_df = add_genres(disliked_song_df, genre_exploded_df)
disliked_genre_count_df = get_genre_counts(genre_exploded_df)
disliked_genre_count_df['genre count'] = disliked_genre_count_df['genre count'] * -1
genre_count_df = pd.concat([liked_genre_count_df, disliked_genre_count_df])
disliked_song_df = establish_genre_score(disliked_genre_count_df, disliked_song_df)
print('\n****PART 3: Combine liked and disliked songs into my_songs dataframe with "Liked" flag and saving as .csv and pickle file***\n')
my_songs_df = combine_dfs_to_create_my_songs_df(liked_song_df,disliked_song_df,'/Users/Will/Documents/Spotipy')
print('\n****PART 4: Get songs from featured playlists and score them based on genre****\n')
raw_featured_playlist_songs_list = get_raw_featured_playlist_song_list()
featured_playlist_song_df = song_metadata_to_df(raw_featured_playlist_songs_list)
featured_playlist_song_df = add_audio_feats(featured_playlist_song_df)
genre_df = get_genres(featured_playlist_song_df)
genre_exploded_df = explode_genres(genre_df)
featured_playlist_song_df = add_genres(featured_playlist_song_df, genre_exploded_df)
featured_playlist_song_df = establish_genre_score(genre_count_df, featured_playlist_song_df)

Scope defined
****PART 1: Get Liked Songs Dataframe****

Get_raw_liked_song_list
Liked song count:  614
>> raw liked songs['0'] 
>> {'added_at': '2020-09-06T04:19:37Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5CAiy5rDr3LaSoGtJOq6A0'}, 'href': 'https://api.spotify.com/v1/artists/5CAiy5rDr3LaSoGtJOq6A0', 'id': '5CAiy5rDr3LaSoGtJOq6A0', 'name': 'Nanci Griffith', 'type': 'artist', 'uri': 'spotify:artist:5CAiy5rDr3LaSoGtJOq6A0'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'S

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:5cNfWI5CJs7ADjSC9SWRPX,The Dustbowl Symphony,5CAiy5rDr3LaSoGtJOq6A0,Nanci Griffith,Love at the Five and Dime,https://open.spotify.com/track/5cNfWI5CJs7ADjS...,14,2020-09-06T04:19:37Z,1999-09-03,1999
spotify:track:7b6fpjQcQcGDfpG7hA3P2k,Austin City Limits Festival,38euc5ce2q2MMzaoJcPstz,Various Artists,It's The Nighttime,https://open.spotify.com/track/7b6fpjQcQcGDfpG...,6,2020-09-06T04:19:36Z,2005-08-30,2005
spotify:track:10e1Jjuo9NviGDjpvJGKIJ,Kamaal The Abstract,3ZotbHeyVQKxQCPDJuQ4SU,Q-Tip,Barely In Love,https://open.spotify.com/track/10e1Jjuo9NviGDj...,40,2020-08-31T22:33:48Z,2009-09-15,2009
spotify:track:5jwSTYsp0KVYYYk97IcPg6,Bottle It In,5gspAQIAH8nJUrMYgXjCJ2,Kurt Vile,Bassackwards,https://open.spotify.com/track/5jwSTYsp0KVYYYk...,56,2020-08-24T16:50:01Z,2018-10-12,2018
spotify:track:6INaSJ4clfZ8pvXSGZQnJ7,"Busted Jukebox, Volume 1",1M3BVQ36cqPQix8lQNCh4K,Shovels & Rope,Unknown Legend,https://open.spotify.com/track/6INaSJ4clfZ8pvX...,59,2020-08-11T14:38:57Z,2015-11-20,2015


Combining song_df with audio_feature_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:5cNfWI5CJs7ADjSC9SWRPX,The Dustbowl Symphony,5CAiy5rDr3LaSoGtJOq6A0,Nanci Griffith,Love at the Five and Dime,https://open.spotify.com/track/5cNfWI5CJs7ADjS...,14,2020-09-06T04:19:37Z,1999-09-03,1999,0.335,...,0.000000,0.241,0.375,67.626,audio_features,5cNfWI5CJs7ADjSC9SWRPX,https://api.spotify.com/v1/tracks/5cNfWI5CJs7A...,https://api.spotify.com/v1/audio-analysis/5cNf...,294667,4
spotify:track:7b6fpjQcQcGDfpG7hA3P2k,Austin City Limits Festival,38euc5ce2q2MMzaoJcPstz,Various Artists,It's The Nighttime,https://open.spotify.com/track/7b6fpjQcQcGDfpG...,6,2020-09-06T04:19:36Z,2005-08-30,2005,0.574,...,0.000013,0.932,0.695,131.563,audio_features,7b6fpjQcQcGDfpG7hA3P2k,https://api.spotify.com/v1/tracks/7b6fpjQcQcGD...,https://api.spotify.com/v1/audio-analysis/7b6f...,243520,4
spotify:track:10e1Jjuo9NviGDjpvJGKIJ,Kamaal The Abstract,3ZotbHeyVQKxQCPDJuQ4SU,Q-Tip,Barely In Love,https://open.spotify.com/track/10e1Jjuo9NviGDj...,40,2020-08-31T22:33:48Z,2009-09-15,2009,0.932,...,0.020900,0.149,0.697,120.018,audio_features,10e1Jjuo9NviGDjpvJGKIJ,https://api.spotify.com/v1/tracks/10e1Jjuo9Nvi...,https://api.spotify.com/v1/audio-analysis/10e1...,243307,1
spotify:track:5jwSTYsp0KVYYYk97IcPg6,Bottle It In,5gspAQIAH8nJUrMYgXjCJ2,Kurt Vile,Bassackwards,https://open.spotify.com/track/5jwSTYsp0KVYYYk...,56,2020-08-24T16:50:01Z,2018-10-12,2018,0.629,...,0.710000,0.178,0.399,95.019,audio_features,5jwSTYsp0KVYYYk97IcPg6,https://api.spotify.com/v1/tracks/5jwSTYsp0KVY...,https://api.spotify.com/v1/audio-analysis/5jwS...,586133,4
spotify:track:6INaSJ4clfZ8pvXSGZQnJ7,"Busted Jukebox, Volume 1",1M3BVQ36cqPQix8lQNCh4K,Shovels & Rope,Unknown Legend,https://open.spotify.com/track/6INaSJ4clfZ8pvX...,59,2020-08-11T14:38:57Z,2015-11-20,2015,0.587,...,0.000006,0.132,0.539,110.936,audio_features,6INaSJ4clfZ8pvXSGZQnJ7,https://api.spotify.com/v1/tracks/6INaSJ4clfZ8...,https://api.spotify.com/v1/audio-analysis/6INa...,268667,4


Getting genres

393  distinct artists
0/393 parsed
100/393 parsed
200/393 parsed
300/393 parsed

genre_df


...

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:3tjFYV6RSFtuktYl3ZtYcq,Mood (feat. Iann Dior),6fWVd57NKTalqvmjRd2t8Z,24kGoldn,Mood (feat. Iann Dior),https://open.spotify.com/track/3tjFYV6RSFtuktY...,96,2020-09-04T04:00:52Z,2020-07-24,2020,0.700,...,0.2720,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4,cali rap
spotify:track:3tjFYV6RSFtuktYl3ZtYcq,Mood (feat. Iann Dior),6fWVd57NKTalqvmjRd2t8Z,24kGoldn,Mood (feat. Iann Dior),https://open.spotify.com/track/3tjFYV6RSFtuktY...,96,2020-09-04T17:46:07Z,2020-07-24,2020,0.700,...,0.2720,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4,cali rap
spotify:track:4Oun2ylbjFKMPTiaSbbCih,WAP (feat. Megan Thee Stallion),4kYSro6naA4h99UJvo89HB,Cardi B,WAP (feat. Megan Thee Stallion),https://open.spotify.com/track/4Oun2ylbjFKMPTi...,100,2020-09-04T04:00:52Z,2020-08-07,2020,0.935,...,0.0824,0.357,133.073,audio_features,4Oun2ylbjFKMPTiaSbbCih,https://api.spotify.com/v1/tracks/4Oun2ylbjFKM...,https://api.spotify.com/v1/audio-analysis/4Oun...,187541,4,pop
spotify:track:4Oun2ylbjFKMPTiaSbbCih,WAP (feat. Megan Thee Stallion),4kYSro6naA4h99UJvo89HB,Cardi B,WAP (feat. Megan Thee Stallion),https://open.spotify.com/track/4Oun2ylbjFKMPTi...,100,2020-09-04T04:00:52Z,2020-08-07,2020,0.935,...,0.0824,0.357,133.073,audio_features,4Oun2ylbjFKMPTiaSbbCih,https://api.spotify.com/v1/tracks/4Oun2ylbjFKM...,https://api.spotify.com/v1/audio-analysis/4Oun...,187541,4,pop rap
spotify:track:4Oun2ylbjFKMPTiaSbbCih,WAP (feat. Megan Thee Stallion),4kYSro6naA4h99UJvo89HB,Cardi B,WAP (feat. Megan Thee Stallion),https://open.spotify.com/track/4Oun2ylbjFKMPTi...,100,2020-09-04T04:00:52Z,2020-08-07,2020,0.935,...,0.0824,0.357,133.073,audio_features,4Oun2ylbjFKMPTiaSbbCih,https://api.spotify.com/v1/tracks/4Oun2ylbjFKM...,https://api.spotify.com/v1/audio-analysis/4Oun...,187541,4,rap



Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,Mood (feat. Iann Dior),6fWVd57NKTalqvmjRd2t8Z,24kGoldn,Mood (feat. Iann Dior),https://open.spotify.com/track/3tjFYV6RSFtuktY...,96,2020-09-04T04:00:52Z,2020-07-24,2020,...,0.2720,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4,NaN
1,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,Mood (feat. Iann Dior),6fWVd57NKTalqvmjRd2t8Z,24kGoldn,Mood (feat. Iann Dior),https://open.spotify.com/track/3tjFYV6RSFtuktY...,96,2020-09-04T17:46:07Z,2020-07-24,2020,...,0.2720,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4,NaN
2,spotify:track:4Oun2ylbjFKMPTiaSbbCih,WAP (feat. Megan Thee Stallion),4kYSro6naA4h99UJvo89HB,Cardi B,WAP (feat. Megan Thee Stallion),https://open.spotify.com/track/4Oun2ylbjFKMPTi...,100,2020-09-04T04:00:52Z,2020-08-07,2020,...,0.0824,0.357,133.073,audio_features,4Oun2ylbjFKMPTiaSbbCih,https://api.spotify.com/v1/tracks/4Oun2ylbjFKM...,https://api.spotify.com/v1/audio-analysis/4Oun...,187541,4,30.0
3,spotify:track:4Oun2ylbjFKMPTiaSbbCih,WAP (feat. Megan Thee Stallion),4kYSro6naA4h99UJvo89HB,Cardi B,WAP (feat. Megan Thee Stallion),https://open.spotify.com/track/4Oun2ylbjFKMPTi...,100,2020-09-04T04:00:52Z,2020-08-07,2020,...,0.0824,0.357,133.073,audio_features,4Oun2ylbjFKMPTiaSbbCih,https://api.spotify.com/v1/tracks/4Oun2ylbjFKM...,https://api.spotify.com/v1/audio-analysis/4Oun...,187541,4,-37.0
4,spotify:track:4Oun2ylbjFKMPTiaSbbCih,WAP (feat. Megan Thee Stallion),4kYSro6naA4h99UJvo89HB,Cardi B,WAP (feat. Megan Thee Stallion),https://open.spotify.com/track/4Oun2ylbjFKMPTi...,100,2020-09-04T04:00:52Z,2020-08-07,2020,...,0.0824,0.357,133.073,audio_features,4Oun2ylbjFKMPTiaSbbCih,https://api.spotify.com/v1/tracks/4Oun2ylbjFKM...,https://api.spotify.com/v1/audio-analysis/4Oun...,187541,4,40.0



Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:00kvKlIvfzQLSi6T2zwXuS,Right On Time,6h3rSZ8VLK7a5vXjEmhfuD,The Brothers Johnson,Strawberry Letter 23,https://open.spotify.com/track/00kvKlIvfzQLSi6...,63,2019-04-23T18:03:40Z,1977-01-01,1977,0.730,...,0.1110,0.752,90.973,audio_features,00kvKlIvfzQLSi6T2zwXuS,https://api.spotify.com/v1/tracks/00kvKlIvfzQL...,https://api.spotify.com/v1/audio-analysis/00kv...,299067,4,20.0
spotify:track:00kvKlIvfzQLSi6T2zwXuS,Right On Time,6h3rSZ8VLK7a5vXjEmhfuD,The Brothers Johnson,Strawberry Letter 23,https://open.spotify.com/track/00kvKlIvfzQLSi6...,63,2020-01-06T16:07:49Z,1977-01-01,1977,0.730,...,0.1110,0.752,90.973,audio_features,00kvKlIvfzQLSi6T2zwXuS,https://api.spotify.com/v1/tracks/00kvKlIvfzQL...,https://api.spotify.com/v1/audio-analysis/00kv...,299067,4,20.0
spotify:track:017PF4Q3l4DBUiWoXk4OWT,Future Nostalgia,6M2wZ9GZgrQXHCFfjv46we,Dua Lipa,Break My Heart,https://open.spotify.com/track/017PF4Q3l4DBUiW...,89,2020-09-04T17:46:07Z,2020-03-27,2020,0.730,...,0.3490,0.467,113.013,audio_features,017PF4Q3l4DBUiWoXk4OWT,https://api.spotify.com/v1/tracks/017PF4Q3l4DB...,https://api.spotify.com/v1/audio-analysis/017P...,221820,4,-14.0
spotify:track:01UYpHuzHi4eB9PAbDoPY2,Seals & Crofts' Greatest Hits,6jdObwsrIjSRnBbMw6lPBj,Seals and Crofts,Summer Breeze,https://open.spotify.com/track/01UYpHuzHi4eB9P...,68,2020-08-17T13:55:31Z,1972,1972,0.490,...,0.0764,0.318,88.718,audio_features,01UYpHuzHi4eB9PAbDoPY2,https://api.spotify.com/v1/tracks/01UYpHuzHi4e...,https://api.spotify.com/v1/audio-analysis/01UY...,205480,4,47.0
spotify:track:022KUgS0Q5c3MTePafMLai,Best of Me,7wLyGTO9vUS7ndlq4BvBGe,Blake Rose,Best of Me,https://open.spotify.com/track/022KUgS0Q5c3MTe...,62,2020-09-03T22:01:00Z,2019-04-19,2019,0.476,...,0.4640,0.247,139.969,audio_features,022KUgS0Q5c3MTePafMLai,https://api.spotify.com/v1/tracks/022KUgS0Q5c3...,https://api.spotify.com/v1/audio-analysis/022K...,213123,4,1.0


In [21]:
my_songs_df = pd.read_pickle("./my_songs.pkl")

In [25]:
create_playlist(pred_like_playlist_name, featured_playlist_song_df, 50)

playlist_df


,playlist_uri,song_count
playlist,,
Genre_Score>=50,spotify:playlist:6xdatpgBLdDUseW9zBHtmt,0
Genre_Score>200,spotify:playlist:7wle5xkDnCYOZHAmbrRyRy,449
Public Songs4,spotify:playlist:368EgihNDeDq7CbABXXrFL,8126
Public Songs3,spotify:playlist:20156WQb2kpmOw3uJnddQt,11000
Public Songs2,spotify:playlist:1pPhACIrG46QrX1YZ4DWV4,11000
Public Songs1,spotify:playlist:5rmu0lDRbuoSc7LSYD033R,11000
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,448


playlist_df


,playlist_uri,song_count
playlist,,
Genre_Score>=50,spotify:playlist:6xdatpgBLdDUseW9zBHtmt,0
Genre_Score>200,spotify:playlist:7wle5xkDnCYOZHAmbrRyRy,449
Public Songs4,spotify:playlist:368EgihNDeDq7CbABXXrFL,8126
Public Songs3,spotify:playlist:20156WQb2kpmOw3uJnddQt,11000
Public Songs2,spotify:playlist:1pPhACIrG46QrX1YZ4DWV4,11000
Public Songs1,spotify:playlist:5rmu0lDRbuoSc7LSYD033R,11000
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,448


In [ ]:
def get_uris(df):
    uri_list = df['uri'].tolist()
    return uri_list

def add_songs_to_playlist(uri_list, playlist_uri):
    df = pd.DataFrame()
    for uri in uri_list:
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={uri}"
        requests.post(url, headers=headers)
    return

def get_uris_raw(uri_list):
    uri_list_split = []
    for uri in uri_list:
        uri_list_split.append(uri.split(':')[2])
    return uri_list_split